# Identify exposures to go into COSMOS coadds

In [2]:
import sys
sys.path.append('../scripts')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skipper import observe
import sigjson_s2022a

In [4]:
_, oiii_pointings = sigjson_s2022a.load_mastercat ()

In [5]:
coo = observe.CopilotOutput ( '../../DataLogs/db_merian.fits', oiii_pointings, skySB_0=22.1 )

In [9]:
in_cosmos = coo.merian_sidecar.apply(lambda x: 'COSMOS' in x['object'], axis=1 )

In [20]:
relevant_columns = ['filename','exptime','mjd_obs','airmass','racenter','deccenter',
                    'rabore','decbore','band','zeropoint','transparency','seeing','sky',
                    'dx','dy','object','t_eff']

In [37]:
# \\ all COSMOS OIII pointings, except the t_eff < 300s exposure(s)
oiii_cosmos = coo.merian_sidecar.loc[in_cosmos].query('t_eff>300.')
assert oiii_cosmos.shape[0] == 40
# \\ DEEP : all pointings
deep = oiii_cosmos[relevant_columns]
# \\ BEST : 4 highest t_eff
best = oiii_cosmos.sort_values('t_eff', ascending=False).iloc[:4][relevant_columns]
# \\ WIDE : 4 with t_eff closest to median
wide = oiii_cosmos.reindex(abs(oiii_cosmos['t_eff'] - np.median(oiii_cosmos['t_eff'])).sort_values ().index[:4])

In [38]:
for name in ['deep','best','wide']:    
    globals()[name].to_csv(f'~/Downloads/oiii_cosmos_{name}.csv')